Coding along - https://www.tensorflow.org/programmers_guide/eager
  
  TensorFlow's eager execution is an imperative programming environment that evaluates operations immediately, without an extra graph-building step.Operations return concrete values instead of constructing a computational graph to run later.  
  This makes it easy to get started with TensorFlow, debug models, reduce boilerplate code, and is fun!


In [1]:
!pip install --upgrade tensorflow

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: protobuf>=3.4.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: six>=1.10.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: grpcio>=1.8.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: tensorboard<1.8.0,>=1.7.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: termcolor>=1.1.0 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: numpy>=1.13.3 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: wheel>=0.26 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: absl-py>=0.1.6 in /usr/local/lib/python3.6/dist-packages (from tensorflow)
Requirement already up-to-date: astor>=0.6.0 in /usr/

Eager execution is a flexible machine learning platform for research and experimentation that provides:

An intuitive interface —Structure your code naturally and use Python data structures. Quickly iterate on small models and small data.  
Easier debugging —Call ops directly to inspect running models and test changes. Use standard Python debugging tools for immediate error reporting.  
Natural control flow —Use Python control flow instead of graph control flow, including support for dynamic models

In [0]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
tf.enable_eager_execution()

To start eager execution, add tf.enable_eager_execution() to the beginning of the program or console session.  
Do not add this operation to other modules that the program calls  
Enabling eager execution changes how TensorFlow operations behave—now they immediately evaluate and return their values to Python.   
tf.Tensor objects reference concrete values instead of symbolic handles to nodes in a computational graph.  
Since there isn't a computational graph to build and run later in a session, it's easy to inspect results using print() or a debugger.   
Evaluating, printing, and checking tensor values does not break the flow for computing gradients

In [6]:
tf.executing_eagerly()
a = tf.constant([[1,2],[3,4]])
print(a)

b = tf.add(a,1)
print(b)

print(a*b)

import numpy as np
c = np.multiply(a,b)
print(c)

print(a.numpy())

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)
[[ 2  6]
 [12 20]]
[[1 2]
 [3 4]]


 TensorFlow math operations convert Python objects and NumPy arrays to tf.Tensor objects  
 

In [7]:
import tensorflow.contrib.eager as tfe

Instructions for updating:
Use the retry module or similar alternatives.


During eager execution, use tfe.GradientTape to trace operations for computing gradients later.  
all forward-pass operations get recorded to a "tape". To compute the gradient, play the tape backwards and then discard.  
A particular tfe.GradientTape can only be computed once, subsequent calls throw a runtime error.</br>  
Automatic Differentition exploits the fact that every computer program, no matter how complicated, executes a sequence of elementary arithmetic operations (addition, subtraction, multiplication, division, etc.)  
and elementary functions (exp, log, sin, cos, etc.). By applying the chain rule repeatedly to these operations, derivatives of arbitrary order can be computed automatically, accurately to working precision, and using at most a small constant factor more arithmetic operations than the original program.

In [11]:
w = tfe.Variable([[1.0]])
with tfe.GradientTape() as tape:
  loss = w * w
  
grad = tape.gradient(loss, [w])
print(grad)

[<tf.Tensor: id=87, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


In [23]:
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])  #Outputs random values from a normal distribution.
noise = tf.random_normal([NUM_EXAMPLES])
#print("training input",training_inputs)
#print("noise",noise)
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
  return input * weight + bias

def loss(weights,biases):
  error = prediction(training_inputs, weights, biases) - training_outputs
  return tf.reduce_mean(tf.square(error))

def grad(weights,biases):
  with tfe.GradientTape() as tape:
    loss_value = loss(weights,biases)
  return tape.gradient(loss_value,[weights,biases])

train_steps = 200
learning_rate = 0.01
W = tfe.Variable([5.]) # random initial value for W
B = tfe.Variable([10.]) # random initial value for B

print("Initial loss: {:.3f}".format(loss(W,B)))

for i in range(train_steps):
  dW, dB = grad(W,B)
  #print("derivatves are",dW,dB)
  W.assign_sub(dW * learning_rate)
  B.assign_sub(dB * learning_rate)
  if i % 20 == 0:
    print("Loss at step {:03d}: {:.3f}".format(i, loss(W,B)))

print("Final loss: {:.3f}".format(loss(W,B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))
  

Initial loss: 68.918
Loss at step 000: 66.252
Loss at step 020: 30.307
Loss at step 040: 14.178
Loss at step 060: 6.938
Loss at step 080: 3.688
Loss at step 100: 2.229
Loss at step 120: 1.574
Loss at step 140: 1.279
Loss at step 160: 1.147
Loss at step 180: 1.087
Final loss: 1.061
W = [3.0121834], B = [2.1231875]
